# SQL Index Guide

We will show how to generate SQL queries on a Snowflake db generated by Airbyte.
This uses the new sql table context str resynthesis.

In [1]:
# Uncomment to enable debugging.

# import logging
# import sys

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Snowflake-SQLAlchemy version fix

Hack to make snowflake-sqlalchemy work despite incompatible sqlalchemy versions

Taken from https://github.com/snowflakedb/snowflake-sqlalchemy/issues/380#issuecomment-1470762025

In [ ]:
# Hack to make snowflake-sqlalchemy work until they patch it

def snowflake_sqlalchemy_20_monkey_patches():
    import sqlalchemy.util.compat

    # make strings always return unicode strings
    sqlalchemy.util.compat.string_types = (str,)
    sqlalchemy.types.String.RETURNS_UNICODE = True

    import snowflake.sqlalchemy.snowdialect

    snowflake.sqlalchemy.snowdialect.SnowflakeDialect.returns_unicode_strings = True

    # make has_table() support the `info_cache` kwarg
    import snowflake.sqlalchemy.snowdialect

    def has_table(self, connection, table_name, schema=None, info_cache=None):
        """
        Checks if the table exists
        """
        return self._has_object(connection, "TABLE", table_name, schema)

    snowflake.sqlalchemy.snowdialect.SnowflakeDialect.has_table = has_table

# usage: call this function before creating an engine:
try:
    snowflake_sqlalchemy_20_monkey_patches()
except Exception as e:
    raise ValueError("Please run `pip install snowflake-sqlalchemy`")

### Define database

We pass the Snowflake uri to the SQL db constructor

In [3]:
snowflake_uri = 'snowflake://<user_login_name>:<password>@<account_identifier>/<database_name>/<schema_name>?warehouse=<warehouse_name>&role=<role_name>'


First we try connecting with sqlalchemy to check the db works.

In [ ]:
from sqlalchemy import select, create_engine, MetaData, Table

# view current table
engine = create_engine(snowflake_uri)
metadata = MetaData(bind=None)
table = Table(
    'ZENDESK_TICKETS', 
    metadata, 
    autoload=True, 
    autoload_with=engine
)
stmt = select(table.columns)


with engine.connect() as connection:
    results = connection.execute(stmt).fetchone()
    print(results)
    print(results.keys())


### Build Index

We then build the SQL Index (`SQLStructStoreIndex`).

In [5]:
from llama_index import SQLStructStoreIndex, SQLDatabase

sql_database = SQLDatabase(engine)
index = SQLStructStoreIndex(
    sql_database=sql_database
)

### Query Index

We first show how we can execute a raw SQL query, which directly executes over the table.

In [6]:
query_engine = index.as_query_engine(
    query_mode="sql"
)
response = query_engine.query("SELECT created_at FROM ZENDESK_TICKETS limit 5")

In [7]:
from IPython.display import Markdown, display
display(Markdown(f"<b>{response}</b>"))

<b>[(datetime.datetime(2022, 7, 18, 16, 59, 13, tzinfo=<UTC>),), (datetime.datetime(2021, 9, 1, 11, 59, 40, tzinfo=<UTC>),), (datetime.datetime(2021, 9, 1, 12, 0, 29, tzinfo=<UTC>),), (datetime.datetime(2022, 7, 18, 15, 22, 39, tzinfo=<UTC>),), (datetime.datetime(2021, 9, 1, 12, 0, 25, tzinfo=<UTC>),)]</b>

We then show a natural language query, which is translated to a SQL query under the hood with our text-to-SQL prompt.

In [8]:
from typing import cast
from llama_index import QueryBundle
from llama_index.indices.struct_store.sql_query import NLStructStoreQueryEngine

# Set resynthesize_table_context to True so the sql engine only tries relevant tables.
query_engine = index.as_query_engine(resynthesize_table_context = True)

nlsqlengine = cast(NLStructStoreQueryEngine, query_engine)
query_str = "When was the last zendesk ticket created?"
display(Markdown(f"<b>{nlsqlengine._synthesize_table_context(QueryBundle(query_str))}</b>"))

response = query_engine.query(query_str)

<b>
Table 'zendesk_tickets' has the relevant column for the query: created_at (TIMESTAMP_TZ). The full schema of the table is: 

Table 'zendesk_tickets' has columns: from_messaging_channel (BOOLEAN), subject (VARCHAR(16777216)), email_cc_ids (VARIANT), created_at (TIMESTAMP_TZ), description (VARCHAR(16777216)), custom_status_id (DECIMAL(38, 0)), external_id (VARCHAR(16777216)), type (VARCHAR(16777216)), via (VARIANT), allow_attachments (BOOLEAN), updated_at (TIMESTAMP_TZ), problem_id (DECIMAL(38, 0)), follower_ids (VARIANT), due_at (TIMESTAMP_TZ), id (DECIMAL(38, 0)), assignee_id (DECIMAL(38, 0)), generated_timestamp (DECIMAL(38, 0)), raw_subject (VARCHAR(16777216)), forum_topic_id (DECIMAL(38, 0)), custom_fields (VARIANT), allow_channelback (BOOLEAN), satisfaction_rating (VARIANT), submitter_id (DECIMAL(38, 0)), priority (VARCHAR(16777216)), collaborator_ids (VARIANT), url (VARCHAR(16777216)), tags (VARIANT), brand_id (DECIMAL(38, 0)), ticket_form_id (DECIMAL(38, 0)), sharing_agreement_ids (VARIANT), group_id (DECIMAL(38, 0)), followup_ids (VARIANT), organization_id (DECIMAL(38, 0)), is_public (BOOLEAN), recipient (VARCHAR(16777216)), has_incidents (BOOLEAN), status (VARCHAR(16777216)), requester_id (DECIMAL(38, 0)), _airbyte_ab_id (VARCHAR(16777216)), _airbyte_emitted_at (TIMESTAMP_TZ), _airbyte_normalized_at (TIMESTAMP_TZ), _airbyte_zendesk_tickets_hashid (VARCHAR(32)), _airbyte_unique_key (VARCHAR(32)) and foreign keys: .</b>

In [9]:
display(Markdown(f"<b>{response}</b>"))

<b> The last Zendesk ticket was created on September 19th, 2022 at 14:53:49 UTC.</b>

In [10]:
# you can also fetch the raw result from SQLAlchemy! 
response.extra_info["result"]

[(datetime.datetime(2022, 9, 19, 14, 53, 49, tzinfo=<UTC>),)]